In [ ]:
import pandas
import re

#### Téléchargement de la dernière version à jour des données extraites

In [ ]:
!wget https://lite.framacalc.org/OKiS1cuixs-codelico.csv -O ../donnees-welcome-guide/OKiS1cuixs-codelico.csv

### Chargement des données

In [ ]:
tableau_complet = pandas.read_csv("../donnees-welcome-guide/OKiS1cuixs-codelico.csv")
#for columns in tableau_complet.columns:   
#    tableau_complet[column] = tableau_complet[column].astype("str")

In [ ]:
colonne_ordre = ["Nom", "adresse"]

In [ ]:
tableau_complet_ordonne = tableau_complet.sort_values(colonne_ordre)

## Tableau des organisations avec services et catégories concaténées dans leurs colonnes respectives

In [ ]:
tableau_complet_ordonne.columns

Sélection des colonnes correspondant aux orgas sans les services (sans "Texte complémentaire Codelico")

In [ ]:
tableau_organisations_colonnes = tableau_complet_ordonne[['Nom', 'adresse', 'Code postal', 'Ville', 'Téléphone', 'Email',
       'Site web', 'Texte complémentaire Codelico', 'Liste de catégories', 'Type','Pays', 'urlImg']]

## Nettoyage des cellules

### Remplacement des valeurs pour les cellules "idem"

Réalisé à la main

Normalisation des noms de villes

In [ ]:
tableau_organisations_ordonne_clean = pandas.DataFrame(columns=tableau_organisations_colonnes.columns)

In [ ]:
tableau_organisations_colonnes.columns

In [ ]:
for column in tableau_organisations_ordonne_clean.columns:
    
    # Retire les espaces en début de cellule - Retire les espaces en fin de cellule - Retire les espaces doublés 
    try:
        tableau_organisations_colonnes[column] = tableau_organisations_colonnes[column].str.replace("^ +", "")\
                                                                 .str.replace(" +$", "")\
                                                                 .str.replace(" +", " ")\
                                                                 .str.replace(";", ",") # Remplace tous les ";" (qui est le séparateur de cellules) par ","
    except:
        print("type error", column)
    

    if column == 'Ville':
        tableau_organisations_colonnes[column] = tableau_organisations_colonnes[column].str.capitalize()\
                                                                                       .str.replace(" +.edex", "") # Retire la mention cedex
            
            # Met la première lettre en majuscule et les autres en minuscules

        tableau_organisations_ordonne_clean[column] = tableau_complet_ordonne_clean_column
    elif column == "Code postal": # Transforme les codes postaux en entiers (retire le ".0")
        tableau_organisations_ordonne_clean[column] = tableau_organisations_colonnes[column].astype("int")
    else:
        tableau_organisations_ordonne_clean[column] = tableau_organisations_colonnes[column]


In [ ]:
tableau_organisations_ordonne_clean.head(2)

### Sélection des organisations dupliquées

Pour chaque organisation dupliquée (même nom, même adresse), stocker les infos et fusionner la liste de catégories et fusionner les services.

Obtention de la liste des organisations dupliquées

In [ ]:
masque_orga_dupliquee = tableau_organisations_ordonne_clean.duplicated(["Nom", "adresse"], keep="first")
tableau_pour_orga_dupliquee_all = tableau_organisations_ordonne_clean[masque_orga_dupliquee]
nom_organisations_dupliquees = tableau_pour_orga_dupliquee_all.Nom

Concaténation des services dans une seule cellule (idem pour les catégories).

In [ ]:
valeurs_pour_orgas_dedupliquees = pandas.DataFrame({}, columns=tableau_organisations_colonnes.columns)

liste_d_index_dupliques_et_geres = []

liste_de_valeurs_pour_orgas_dedupliquees = []

for organisation in nom_organisations_dupliquees:    
    # Sélection des lignes correspondant à une seule organisation (dont on sait qu'elle est dupliquée)
    masque_organisation_est_dupliquee = tableau_organisations_colonnes.Nom == organisation
    tableau_pour_orga_dupliquee = tableau_organisations_ordonne_clean[masque_organisation_est_dupliquee]
        
    # Conservation en mémoire des lignes qui sont dans le tableau des organisations dupliquées
    for index in tableau_pour_orga_dupliquee.index.values:
        liste_d_index_dupliques_et_geres.append(index)

    # Concaténation de la liste de services
    liste_de_services = []
    for service in tableau_pour_orga_dupliquee["Texte complémentaire Codelico"]:  
        liste_de_services.append(str(service) + "///") 
        
    # Concaténation de la liste de catégories
    liste_de_categories = []
    liste_de_categories_dupliquees = []
    for categories in tableau_pour_orga_dupliquee["Liste de catégories"]:
        categories = str(categories)
        liste_de_categories_pour_cette_orga_a_nettoyer = categories.split(", ")
        
        liste_de_categories_dupliquees_pour_cette_orga = []
        for categorie_a_nettoyer in liste_de_categories_pour_cette_orga_a_nettoyer:
            
            categorie = re.sub(r" *$", "", re.sub(r"^ *", "", categorie_a_nettoyer))
        
            liste_de_categories_dupliquees_pour_cette_orga.append(categorie)
              
        liste_de_categories_dupliquees += liste_de_categories_dupliquees_pour_cette_orga        
    liste_de_categories = list(set(liste_de_categories_dupliquees))
    
    # Pour une orga dupliquée, sélection d'une seule ligne, peu importe les valeurs de service et catégories
    valeurs_pour_orga = tableau_pour_orga_dupliquee.head(1)
    
    # Stockage de la liste des services et de la liste des catégories dans cette ligne
    valeurs_pour_orga["Liste de catégories"] = ", ".join(liste_de_categories)
    valeurs_pour_orga["Texte complémentaire Codelico"] = ", ".join(liste_de_services)
        
    # Ajout de la ligne avec les services concaténés et catégories concaténées dans un tableau
    liste_de_valeurs_pour_orgas_dedupliquees.append(valeurs_pour_orga)

valeurs_pour_orgas_dedupliquees = pandas.concat(liste_de_valeurs_pour_orgas_dedupliquees).drop_duplicates()

### Sélection des lignes des organisations non dupliquées

Il s'agit de toutes les autres lignes que celles dont l'indice a été vu dans la gestion des organisations dupliquées.

In [ ]:
valeurs_pour_orgas_non_dupliquees = tableau_organisations_ordonne_clean[tableau_organisations_colonnes.columns].drop(liste_d_index_dupliques_et_geres)

### Création d'un tableau avec les organisations non dupliquées et les organisations dupliquées

In [ ]:
valeurs_pour_orgas = valeurs_pour_orgas_dedupliquees.append(valeurs_pour_orgas_non_dupliquees)

Ce tableau contient une ligne par organisation, avec les services placés dans une seule cellule. Idem pour les catégories.

Dans un premier temps, l'import ne fonctionne pas.

## Normalisation des codes postaux à une valeur unique par ville

Communecter ne supporte pas actuellement les codes postaux avec le niveau de précision donné par la mairie (présence du cedex dans le code ?).

Pour chaque ville, un code postal est associé.

In [ ]:
liste_de_villes = valeurs_pour_orgas[["Ville"]].sort_values(by="Ville").drop_duplicates()

Pour certaines villes, il y a plusieurs codes postaux

In [ ]:
for ville in liste_de_villes.values:
    masque_ville = valeurs_pour_orgas.Ville == ville[0]
    print(ville, list(set(valeurs_pour_orgas[masque_ville]["Code postal"].values)))

remplacement du dernier chiffre par "0" pour retirre ce supplément d'information.

In [ ]:
for ville in liste_de_villes.values:
    masque_ville = valeurs_pour_orgas.Ville == ville[0]
    codes_postaux = list(set(valeurs_pour_orgas[masque_ville]["Code postal"].values))
    if len(codes_postaux) > 1:        
        codes_postaux_normalises = []
        for code_postal in codes_postaux:
            codes_postaux_normalises.append(re.sub(r".$", "0", str(code_postal)))
        print(ville, list(set(codes_postaux_normalises)))

Seule la ville de Strasbourg a d'autres précisions au niveau de l'avant dernier chiffre

In [ ]:
for ville in liste_de_villes.values:
    masque_ville = valeurs_pour_orgas.Ville == ville[0]
    codes_postaux = list(set(valeurs_pour_orgas[masque_ville]["Code postal"].values))
    if len(codes_postaux) > 1:        
        codes_postaux_normalises = []
        for code_postal in codes_postaux:
            codes_postaux_normalises.append(re.sub(r"..$", "00", str(code_postal)))
        print(ville, list(set(codes_postaux_normalises)))

Remplacement dans le tableau de données

In [ ]:
valeurs_pour_orgas["Code postal"] = valeurs_pour_orgas["Code postal"].astype("str").str.replace(r"..$", "00")

In [ ]:
valeurs_pour_orgas.head().tail(1)["Texte complémentaire Codelico"].values

Écriture du fichier

In [ ]:
valeurs_pour_orgas.to_csv("../donnees-welcome-guide/une-organisation-services-concatenes.csv", sep=";", index=None)

Versionnement et mise à disposition sur github

In [ ]:
!git add ../donnees-welcome-guide/une-organisation-services-concatenes.csv && git commit -m "Met à jour les données traitées"

## Import dans Communecter

Via admin public avec mon compte.

Conversion du CSV vers le JSON validé

![](../images/config-admin-public-codelicoconcatene20190622.png)

In [ ]:
clef = "codelicoconcatene20190622"

Import du JSON validé.

Le fichier est dans une archive zip

In [ ]:
!ls ../donnees-welcome-guide/une-organisation-services-concatenes_Import.zip

In [ ]:
!unzip ../donnees-welcome-guide/une-organisation-services-concatenes_Import.zip -d ../donnees-welcome-guide/

In [ ]:
import json

In [ ]:
dictionnaire_pret_a_importer = json.load(open("../donnees-welcome-guide/une-organisation-services-concatenes_StandardForCommunecter.json", 'r'))

In [ ]:
len(dictionnaire_pret_a_importer)